In [2]:
import pandas as pd

from poandy.controller.order import OrderController
from poandy.controller.position import PositionController
from poandy.controller.account import AccountController
from poandy.util.utils import Utils

import time

The bid price represents the maximum price that a buyer is willing to pay for a unit of X. The ask price represents the minimum price that a seller is willing to take for that same unit of X.

In [3]:
account_id = AccountController.get_default_account_id()

In [4]:
def buy(account_id):
    return OrderController.create_order(
        account_id,
        "MARKET",
        "100000",
        "EUR_USD",
        "FOK",
        "DEFAULT"
    )

In [5]:
def long(account_id):
    return PositionController.close_instrument_positions(account_id, "EUR_USD", long_units="ALL")

Sample buy and close:

In [6]:
buy(account_id)

{'orderCreateTransaction': {'type': 'MARKET_ORDER',
  'instrument': 'EUR_USD',
  'units': '100000',
  'timeInForce': 'FOK',
  'positionFill': 'DEFAULT',
  'reason': 'CLIENT_ORDER',
  'id': '12',
  'accountID': '101-003-16386799-001',
  'userID': 16386799,
  'batchID': '12',
  'requestID': '42737675088489848',
  'time': '1599521176.879484958'},
 'orderFillTransaction': {'type': 'ORDER_FILL',
  'orderID': '12',
  'instrument': 'EUR_USD',
  'units': '100000',
  'requestedUnits': '100000',
  'price': '1.18187',
  'pl': '0.0000',
  'financing': '0.0000',
  'commission': '0.0000',
  'accountBalance': '99889.9629',
  'gainQuoteHomeConversionFactor': '1.35960780',
  'lossQuoteHomeConversionFactor': '1.37327220',
  'guaranteedExecutionFee': '0.0000',
  'halfSpreadCost': '5.4658',
  'fullVWAP': '1.18187',
  'reason': 'MARKET_ORDER',
  'tradeOpened': {'price': '1.18187',
   'tradeID': '13',
   'units': '100000',
   'guaranteedExecutionFee': '0.0000',
   'halfSpreadCost': '5.4658',
   'initialMarg

In [7]:
long(account_id)

{'longOrderCreateTransaction': {'type': 'MARKET_ORDER',
  'instrument': 'EUR_USD',
  'units': '-100000',
  'timeInForce': 'FOK',
  'positionFill': 'REDUCE_ONLY',
  'reason': 'POSITION_CLOSEOUT',
  'longPositionCloseout': {'instrument': 'EUR_USD', 'units': 'ALL'},
  'id': '14',
  'accountID': '101-003-16386799-001',
  'userID': 16386799,
  'batchID': '14',
  'requestID': '42737675092685576',
  'time': '1599521177.632469064'},
 'longOrderFillTransaction': {'type': 'ORDER_FILL',
  'orderID': '14',
  'instrument': 'EUR_USD',
  'units': '-100000',
  'requestedUnits': '-100000',
  'price': '1.18179',
  'pl': '-10.9862',
  'financing': '0.0000',
  'commission': '0.0000',
  'accountBalance': '99878.9767',
  'gainQuoteHomeConversionFactor': '1.35960780',
  'lossQuoteHomeConversionFactor': '1.37327220',
  'guaranteedExecutionFee': '0.0000',
  'halfSpreadCost': '5.4658',
  'fullVWAP': '1.18179',
  'reason': 'MARKET_ORDER_POSITION_CLOSEOUT',
  'tradesClosed': [{'tradeID': '13',
    'units': '-1000

In [8]:
res = []

In [9]:
for i in range(3600):
    pair = {}
    
    try:
        pair['latency_buy'] = -Utils.get_unix_timestamp()
        buy_response = buy(account_id)['orderFillTransaction']
        pair['latency_buy'] += Utils.get_unix_timestamp()
        pair['buy_price'] = buy_response['tradeOpened']['price']
        pair['buy_hsc'] = buy_response['tradeOpened']['halfSpreadCost']
        pair['buy_bid'] = buy_response['fullPrice']['closeoutBid']
        pair['buy_ask'] = buy_response['fullPrice']['closeoutAsk']

        pair['latency_close'] = -Utils.get_unix_timestamp()
        close_response = long(account_id)['longOrderFillTransaction']
        pair['latency_close'] += Utils.get_unix_timestamp()
        pair['close_price'] = close_response['tradesClosed'][0]['price']
        pair['close_hsc'] = close_response['tradesClosed'][0]['halfSpreadCost']
        pair['close_bid'] = close_response['fullPrice']['closeoutBid']
        pair['close_ask'] = close_response['fullPrice']['closeoutAsk']
    except Exception as e:
        print(e)
        continue
    
    res.append(pair)
    time.sleep(1)

Transaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction'
'orderFillTransaction

In [10]:
df = pd.DataFrame(res)

In [11]:
df.to_csv("slippage_spread_oanda.csv", index=False)

In [12]:
len(df)/3600

0.4386111111111111

56% of orders failed out :(